In [39]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import mixture
sns.set(style='whitegrid')
sns.set_palette('bright')
import warnings
warnings.filterwarnings('ignore')
import collections
from scipy.spatial import distance


In [40]:
df = pd.read_csv('preci_alt.csv')
df.columns = ['date','value']
df.date = pd.to_datetime(df.date,format='%b %d, %Y')
df.head(3)

,date,value
0,2010-01-01,0.0
1,2010-01-02,0.0
2,2010-01-03,0.0


In [41]:
df19 = df[df.date.dt.year==2019]

In [42]:
def gmm2k(q1,q2):
    lowest_bic = np.infty
    bic = []
    n_components_range = range(1, 8)
    cv_types = ['spherical', 'tied', 'diag', 'full']
    for cv_type in cv_types:
        for n_components in n_components_range:
            # Fit a Gaussian mixture with EM
            gmm = mixture.GaussianMixture(n_components=n_components,
                                          covariance_type=cv_type)
            q1 = np.asarray(q1)
            q1 =q1.reshape(-1,1)
            gmm.fit(q1)
            bic.append(gmm.bic(q1))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmmq1 = gmm
    lowest_bic = np.infty
    bic = []
    n_components_range = range(1, 8)
    cv_types = ['spherical', 'tied', 'diag', 'full']
    for cv_type in cv_types:
        for n_components in n_components_range:
            # Fit a Gaussian mixture with EM
            gmm = mixture.GaussianMixture(n_components=n_components,
                                          covariance_type=cv_type)
            q2 = np.asarray(q2)
            q2 =q2.reshape(-1,1)
            gmm.fit(q2)
            bic.append(gmm.bic(q2))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmmq2 = gmm
    mins = min(best_gmmq1.n_components,best_gmmq2.n_components)
    if best_gmmq1.n_components > best_gmmq2.n_components:
        lowest_bic = np.infty
        bic = []
        cv_types = ['spherical', 'tied', 'diag', 'full']
        for cv_type in cv_types:
            # Fit a Gaussian mixture with EM
            gmm = mixture.GaussianMixture(n_components=mins,
                                              covariance_type=cv_type)
            q1 = np.asarray(q1)
            q1 =q1.reshape(-1,1)
            gmm.fit(q1)
            bic.append(gmm.bic(q1))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmmq1 = gmm
    else:
        lowest_bic = np.infty
        bic = []
        cv_types = ['spherical', 'tied', 'diag', 'full']
        for cv_type in cv_types:
            # Fit a Gaussian mixture with EM
            gmm = mixture.GaussianMixture(n_components=mins,
                                              covariance_type=cv_type)
            q2 = np.asarray(q2)
            q2 =q2.reshape(-1,1)
            gmm.fit(q2)
            bic.append(gmm.bic(q2))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmmq2 = gmm
    #print(best_gmmq1.get_params())
    #print(best_gmmq2.get_params())
    #print(best_gmmq1.means_)
    #print(best_gmmq2.means_)
    #print(best_gmmq1.weights_)
    #print(best_gmmq2.weights_)
    means=[]
    for tt in range(len(best_gmmq1.means_)):
        means.append(best_gmmq1.means_[tt][0])
    x = dict(zip(best_gmmq1.weights_,means))
    od1 = collections.OrderedDict(sorted(x.items(),reverse=True))
    od1 = list(zip(list(od1.keys()),list(od1.values())))
    means=[]
    for tt in range(len(best_gmmq2.means_)):
        means.append(best_gmmq2.means_[tt][0])
    y = dict(zip(best_gmmq2.weights_,means))
    od2 = collections.OrderedDict(sorted(y.items(),reverse=True))
    od2 = list(zip(list(od2.keys()),list(od2.values())))
    print(od1)
    print(od2)
    diffmean = []
    diffweights = []
    for tt in range(len(od2)):
        diffmean.append(od1[tt][1] - od2[tt][1])
        diffweights.append(od1[tt][0] - od2[tt][0])    
    r=[]
    for i in range(0,len(diffmean)):
        r.append(diffmean[i] * diffweights[i])
    return od1,od2,sum(r)

In [43]:
df19j = df19[df19.date.dt.month == 7]
df19f = df19[df19.date.dt.month == 6]
qw,we,r= gmm2k(df19j.value,df19f.value)
print(qw)
print(we)
print(r)

[(0.932444691104159, 0.009094296424982948), (0.06755530889584142, 0.03682632135568748)]
[(0.9324253014464888, 0.003844462857303236), (0.06757469855351099, 0.05103476550445302)]
[(0.932444691104159, 0.009094296424982948), (0.06755530889584142, 0.03682632135568748)]
[(0.9324253014464888, 0.003844462857303236), (0.06757469855351099, 0.05103476550445302)]
3.7728934376413665e-07


In [44]:
a = np.zeros(shape=(12, 12))
for i in range(1,13):
    for j in range(1,13):
        dfc = df19[df19.date.dt.month == i]
        dfd = df19[df19.date.dt.month == j]
        _,_,r = gmm2k(dfc.value,dfd.value)
        a[i-1][j-1] = r
a

[(1.0000000000000002, 0.0)]
[(1.0000000000000002, 0.0)]
[(1.0000000000000002, 0.0)]
[(1.0000000000000002, 0.0)]
[(1.0000000000000002, 0.0)]
[(1.0000000000000002, 0.0)]
[(1.0000000000000002, 0.0)]
[(1.0000000000000002, 0.0003666666666666666)]
[(1.0000000000000002, 0.0)]
[(1.0000000000000002, 0.0)]
[(1.0000000000000002, 0.0)]
[(1.0000000000000002, 0.007033333333333334)]
[(1.0000000000000002, 0.0)]
[(1.0000000000000002, 0.01096774193548387)]
[(1.0000000000000002, 0.0)]
[(1.0000000000000002, 0.014387096774193546)]
[(1.0000000000000002, 0.0)]
[(1.0000000000000002, 0.011066666666666667)]
[(1.0000000000000002, 0.0)]
[(1.0000000000000002, 0.006548387096774193)]
[(1.0000000000000002, 0.0)]
[(1.0000000000000002, 0.0005666666666666666)]
[(1.0000000000000002, 0.0)]
[(1.0000000000000002, 0.0004827586206896552)]
[(1.0000000000000002, 0.0)]
[(1.0000000000000002, 0.0)]
[(1.0000000000000002, 0.0)]
[(1.0000000000000002, 0.0)]
[(1.0000000000000002, 0.0)]
[(1.0000000000000002, 0.0)]
[(1.0000000000000002, 

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.0000000

In [45]:
dfw = pd.DataFrame(a)
cm = sns.light_palette("green", as_cmap=True)
s = dfw.style.background_gradient(cmap=cm)
s

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,-0.013100,0.000013,0.000751,-0.009806,0.000663,-0.001356,-0.001070
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.001508,-0.003205,-0.004962,-0.000729,-0.000591
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000751,0.001526,0.000000,-0.013544,0.002858,0.000571,0.001039
8,0.000000,0.000000,0.000000,0.000000,0.000000,-0.009806,-0.003205,-0.013544,0.000000,0.000431,0.004093,0.004624
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000618,-0.004846,0.002930,0.000478,-0.000000,0.001359,0.001748


In [7]:
print(best_gmmq1.means_)
print(best_gmmq2.means_)
print(best_gmmq1.weights_)
print(best_gmmq2.weights_)

[[ 34.36252567]
 [300.04340106]]
[[272.25874282]
 [ 75.06318767]]
[0.33332858 0.66667142]
[0.32922962 0.67077038]


## OrderbyMeans

In [8]:
means=[]
for i in range(len(best_gmmq1.means_)):
    means.append(best_gmmq1.means_[i][0])
x = dict(zip(means,best_gmmq1.weights_))
od1 = collections.OrderedDict(sorted(x.items(),reverse=True))
od1

OrderedDict([(300.04340106175016, 0.6666714159661873),
             (34.362525668641766, 0.3333285840338131)])

In [9]:
means=[]
for i in range(len(best_gmmq2.means_)):
    means.append(best_gmmq2.means_[i][0])
x = dict(zip(means,best_gmmq2.weights_))
od2 = collections.OrderedDict(sorted(x.items(),reverse=True))
od2

OrderedDict([(272.25874282362764, 0.32922962293822033),
             (75.06318767078143, 0.6707703770617822)])

## OrderbyWeights

In [10]:
means=[]
for i in range(len(best_gmmq1.means_)):
    means.append(best_gmmq1.means_[i][0])
x = dict(zip(best_gmmq1.weights_,means))
od1 = collections.OrderedDict(sorted(x.items(),reverse=True))
od1 = list(zip(list(od1.keys()),list(od1.values())))
od1

[(0.6666714159661873, 300.04340106175016),
 (0.3333285840338131, 34.362525668641766)]

In [11]:
means=[]
for i in range(len(best_gmmq2.means_)):
    means.append(best_gmmq2.means_[i][0])
y = dict(zip(best_gmmq2.weights_,means))
od2 = collections.OrderedDict(sorted(y.items(),reverse=True))
od2 = list(zip(list(od2.keys()),list(od2.values())))
od2

[(0.6707703770617822, 75.06318767078143),
 (0.32922962293822033, 272.25874282362764)]

In [12]:
thres = 0.05
diffmean = []
diffweights = []
for i in range(len(od1)):
    diffmean.append(od1[i][1] - od2[i][1])
    diffweights.append(od1[i][0] - od2[i][0])        

In [13]:
print(diffmean)
print(diffweights)

[224.98021339096874, -237.8962171549859]
[-0.004098961095594933, 0.004098961095592768]
